# 1. 예제

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD



In [ ]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('샘플의 수 :',len(documents))


샘플의 수 : 11314


In [ ]:
documents[1]


"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [ ]:
documents = documents[:100]

In [ ]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())


In [ ]:
news_df['clean_doc'].head()

0    well sure about story seem biased. what disagr...
1    yeah, expect people read faq, etc. actually ac...
2    although realize that principle your strongest...
3    notwithstanding legitimate fuss about this pro...
4    well, will have change scoring playoff pool. u...
Name: clean_doc, dtype: object

In [ ]:
news_df['clean_doc'][0]

'well sure about story seem biased. what disagree with your statement that u.s. media ruin israels reputation. that rediculous. u.s. media most pro-israeli media world. having lived europe realize that incidences such described letter have occured. u.s. media whole seem ignore them. u.s. subsidizing israels existance europeans least same degree). think that might reason they report more clearly atrocities. what shame that austria, daily reports inhuman acts commited israeli soldiers blessing received from government makes some holocaust guilt away. after all, look jews treating other races when they power. unfortunate.'

In [ ]:
import nltk
nltk.download('stopwords')
# NLTK로부터 불용어를 받아온다.
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거합니다.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# 역토큰화 (토큰화 작업을 역으로 되돌림)
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc


In [ ]:
news_df['clean_doc'].head()

0    well sure story seem biased. disagree statemen...
1    yeah, expect people read faq, etc. actually ac...
2    although realize principle strongest points, w...
3    notwithstanding legitimate fuss proposal, much...
4    well, change scoring playoff pool. unfortunate...
Name: clean_doc, dtype: object

In [ ]:
type(news_df['clean_doc'])

pandas.core.series.Series

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', max_features= 10, # 상위 1,000개의 단어를 보존
max_df = 0.5, smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)


TF-IDF 행렬의 크기 : (100, 10)


In [ ]:
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)


# 2. 실전

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 이것만 돌리기

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import json

path = '/content/drive/MyDrive/데이터/오리지널/'

df_train = pd.read_csv(path+'train_data.csv')
df_val = pd.read_csv(path+'val_data.csv')
df_test = pd.read_csv(path+'test_data.csv')

with open(path+'label_mapping.json', 'r') as f:
    label_mapping = json.load(f)

label_encoder = LabelEncoder()
label_encoder.classes_ = np.array(list(label_mapping.keys()))

In [ ]:
df_train.head(2)

,document,press_encoded
0,서울 5일 서울 중구 하나은행 위변조대응센터에서 직원이 달러를 정리하고 있다. 최...,0
1,기사내용 요약 무역적자·경기침체 영향 마이크론發 반도체 투심 악화…삼전 매물 쏟아져...,1


In [ ]:
df_train.shape

(6717, 2)

In [ ]:
df_train.loc[5991,'document']

' 다음은 1일 장 마감 후 주요 종목 뉴스다.'

In [ ]:
!pip install pyLDAvis konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 44.8 MB/s eta 0:00:00


In [ ]:
import konlpy
from konlpy.tag import Okt
import requests

stopwords_url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ko/master/stopwords-ko.txt"
korean_stopwords = requests.get(stopwords_url).text.split('\n')
korean_stopwords = set([word.strip() for word in korean_stopwords if word.strip()])

okt = Okt()

def tokenizer(text):
    tokens = okt.morphs(text)
    words = [word.lower() for word in tokens if word not in korean_stopwords and len(word) > 2]
    return words

In [ ]:
from tqdm import tqdm

texts_train = [tokenizer(news) for news in tqdm(df_train['document'], desc="Tokenizing Train Data")]
texts_val = [tokenizer(news) for news in tqdm(df_val['document'], desc="Tokenizing Val Data")]
texts_test = [tokenizer(news) for news in tqdm(df_test['document'], desc="Tokenizing Test Data")]

Tokenizing Test Data: 100%|██████████| 1440/1440 [01:01<00:00, 23.52it/s]


In [ ]:
len(texts_train), len(texts_val), len(texts_test)

(6717, 1439, 1440)

In [ ]:
path = '/content/drive/MyDrive/데이터/'

# 텍스트 파일로 저장
with open(path+'texts_train.txt', 'w', encoding='utf-8') as f:
    for sublist in texts_train:
        line = " ".join(sublist)
        f.write(line + "\n")
with open(path+'texts_val.txt', 'w', encoding='utf-8') as f:
    for sublist in texts_val:
        line = " ".join(sublist)
        f.write(line + "\n")
with open(path+'texts_test.txt', 'w', encoding='utf-8') as f:
    for sublist in texts_test:
        line = " ".join(sublist)
        f.write(line + "\n")

print("텍스트 파일로 저장되었습니다.")

텍스트 파일로 저장되었습니다.


In [ ]:
# # 불러오기

# texts_train = []
# with open(path+'texts_train.txt', 'r', encoding='utf-8') as f:
#     for line in f:
#         # 각 줄을 공백을 기준으로 분할하여 리스트로 변환
#         sublist = line.strip().split()
#         texts_train.append(sublist)

# texts_val = []
# with open(path+'texts_val.txt', 'r', encoding='utf-8') as f:
#     for line in f:
#         # 각 줄을 공백을 기준으로 분할하여 리스트로 변환
#         sublist = line.strip().split()
#         texts_val.append(sublist)

# texts_test = []
# with open(path+'texts_test.txt', 'r', encoding='utf-8') as f:
#     for line in f:
#         # 각 줄을 공백을 기준으로 분할하여 리스트로 변환
#         sublist = line.strip().split()
#         texts_test.append(sublist)

# print("불러왔다.")

불러왔다.


In [ ]:
texts = texts_train+texts_val
len(texts_train), len(texts_val), len(texts_test)

(6717, 1439, 1440)

In [ ]:
len(texts_train), texts_train[0]

(6717,
 ['하나은행',
  '외환시장',
  '우리나라',
  '94억달러',
  '줄었다',
  '2008년',
  '글로벌',
  '금융위기',
  '한국은행',
  '따르면',
  '우리나라',
  '382억',
  '8천만달러',
  '477억',
  '1천만달러',
  '94억',
  '3천만달러'])

In [ ]:
# 텍스트 파일로 저장
with open(path+'multi_list.txt', 'w', encoding='utf-8') as f:
    for sublist in texts:
        line = " ".join(sublist)
        f.write(line + "\n")

print("텍스트 파일로 저장되었습니다.")

텍스트 파일로 저장되었습니다.


In [ ]:
# # 텍스트 파일에서 불러오기
# loaded_data = []

# with open(path+'multi_list.txt', 'r', encoding='utf-8') as f:
#     for line in f:
#         # 각 줄을 공백을 기준으로 분할하여 리스트로 변환
#         sublist = line.strip().split()
#         loaded_data.append(sublist)

# print("불러온 데이터:", loaded_data)

# texts = loaded_data
# loaded_data[0]

불러온 데이터: 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



['하나은행',
 '외환시장',
 '우리나라',
 '94억달러',
 '줄었다',
 '2008년',
 '글로벌',
 '금융위기',
 '한국은행',
 '따르면',
 '우리나라',
 '382억',
 '8천만달러',
 '477억',
 '1천만달러',
 '94억',
 '3천만달러']

In [ ]:
#데이터프레임 생성
import pandas as pd

# 문자열로 변환된 리스트
data_strings = [" ".join(sublist) for sublist in texts]

# 데이터프레임 생성
news_df = pd.DataFrame(data_strings, columns=['clean_doc'])

# 인덱스 설정 (생략 가능, 기본 인덱스는 0부터 시작)
news_df.index = range(len(news_df))

news_df['document'] = list(range(len(texts_train))) + list(range(len(texts_val)))
news_df['data type'] = ['train'] * len(texts_train) + ['val'] * len(texts_val)

news_df.head()

,clean_doc,document,data type
0,하나은행 외환시장 우리나라 94억달러 줄었다 2008년 글로벌 금융위기 한국은행 따...,0,train
1,무역적자 마이크론 반도체 쏟아져 하나은행 딜링룸 코스피 22.02 포인트 0.94%...,1,train
2,김근수 비트코인 서초구 전광판 나타나고 2022.07 com 받습니다 뉴시스 소중한...,2,train
3,에어쇼 항공기 난이도 8800 블랙이글스 항공기 에어쇼 블랙이글스 black eag...,3,train
4,화물차 줄여주기 보조금,4,train


In [ ]:
import pandas as pd

# 데이터프레임을 CSV 파일로 저장
csv_file_path = path+'news_df.csv'
news_df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')
print(f"\n데이터프레임이 {csv_file_path} 파일로 저장되었습니다.")


데이터프레임이 /content/drive/MyDrive/데이터/news_df.csv 파일로 저장되었습니다.


In [ ]:
# # CSV 파일에서 데이터프레임 불러오기

# csv_file_path = path+'news_df.csv'
# news_df = pd.read_csv(csv_file_path, encoding='utf-8-sig')

# print("\nCSV 파일에서 불러온 데이터프레임:")
# print(df_loaded.head())


CSV 파일에서 불러온 데이터프레임:
                                           clean_doc  document data type
0  하나은행 외환시장 우리나라 94억달러 줄었다 2008년 글로벌 금융위기 한국은행 따...         0     train
1  무역적자 마이크론 반도체 쏟아져 하나은행 딜링룸 코스피 22.02 포인트 0.94%...         1     train
2  김근수 비트코인 서초구 전광판 나타나고 2022.07 com 받습니다 뉴시스 소중한...         2     train
3  에어쇼 항공기 난이도 8800 블랙이글스 항공기 에어쇼 블랙이글스 black eag...         3     train
4                                       화물차 줄여주기 보조금         4     train


## 여기서부터 다시 시작

### 데이터 불러오기

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import json

path = '/content/drive/MyDrive/데이터/오리지널/'

df_train = pd.read_csv(path+'train_data.csv')
df_val = pd.read_csv(path+'val_data.csv')
df_test = pd.read_csv(path+'test_data.csv')

with open(path+'label_mapping.json', 'r') as f:
    label_mapping = json.load(f)

label_encoder = LabelEncoder()
label_encoder.classes_ = np.array(list(label_mapping.keys()))

In [ ]:
# 불러오기

path = '/content/drive/MyDrive/데이터/'

texts_train = []
with open(path+'texts_train.txt', 'r', encoding='utf-8') as f:
    for line in f:
        # 각 줄을 공백을 기준으로 분할하여 리스트로 변환
        sublist = line.strip().split()
        texts_train.append(sublist)

texts_val = []
with open(path+'texts_val.txt', 'r', encoding='utf-8') as f:
    for line in f:
        # 각 줄을 공백을 기준으로 분할하여 리스트로 변환
        sublist = line.strip().split()
        texts_val.append(sublist)

texts_test = []
with open(path+'texts_test.txt', 'r', encoding='utf-8') as f:
    for line in f:
        # 각 줄을 공백을 기준으로 분할하여 리스트로 변환
        sublist = line.strip().split()
        texts_test.append(sublist)

print("불러왔다.")

불러왔다.


In [ ]:
# 텍스트 파일에서 불러오기
loaded_data = []

with open(path+'multi_list.txt', 'r', encoding='utf-8') as f:
    for line in f:
        # 각 줄을 공백을 기준으로 분할하여 리스트로 변환
        sublist = line.strip().split()
        loaded_data.append(sublist)

print("불러온 데이터:", loaded_data)

texts = loaded_data
loaded_data[0]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



['하나은행',
 '외환시장',
 '우리나라',
 '94억달러',
 '줄었다',
 '2008년',
 '글로벌',
 '금융위기',
 '한국은행',
 '따르면',
 '우리나라',
 '382억',
 '8천만달러',
 '477억',
 '1천만달러',
 '94억',
 '3천만달러']

In [ ]:
#데이터프레임 생성
import pandas as pd

# 문자열로 변환된 리스트
data_strings = [" ".join(sublist) for sublist in texts]

# 데이터프레임 생성
news_df = pd.DataFrame(data_strings, columns=['clean_doc'])

# 인덱스 설정 (생략 가능, 기본 인덱스는 0부터 시작)
news_df.index = range(len(news_df))

news_df['document'] = list(range(len(texts_train))) + list(range(len(texts_val)))
news_df['data type'] = ['train'] * len(texts_train) + ['val'] * len(texts_val)

news_df.head()

,clean_doc,document,data type
0,하나은행 외환시장 우리나라 94억달러 줄었다 2008년 글로벌 금융위기 한국은행 따...,0,train
1,무역적자 마이크론 반도체 쏟아져 하나은행 딜링룸 코스피 22.02 포인트 0.94%...,1,train
2,김근수 비트코인 서초구 전광판 나타나고 2022.07 com 받습니다 뉴시스 소중한...,2,train
3,에어쇼 항공기 난이도 8800 블랙이글스 항공기 에어쇼 블랙이글스 black eag...,3,train
4,화물차 줄여주기 보조금,4,train


In [ ]:
import copy

df_loaded = copy.deepcopy(news_df.clean_doc)
df_loaded.head()

0    하나은행 외환시장 우리나라 94억달러 줄었다 2008년 글로벌 금융위기 한국은행 따...
1    무역적자 마이크론 반도체 쏟아져 하나은행 딜링룸 코스피 22.02 포인트 0.94%...
2    김근수 비트코인 서초구 전광판 나타나고 2022.07 com 받습니다 뉴시스 소중한...
3    에어쇼 항공기 난이도 8800 블랙이글스 항공기 에어쇼 블랙이글스 black eag...
4                                         화물차 줄여주기 보조금
Name: clean_doc, dtype: object

In [ ]:
remove_idx_list = df_loaded[df_loaded.apply(lambda x:len(x)==0)].index.tolist()
remove_idx_list

[411,
 829,
 1235,
 1419,
 3020,
 3225,
 3706,
 3856,
 5991,
 6374,
 6610,
 7142,
 7358,
 7636]

In [ ]:
df_train.loc[remove_idx_list[:-3],'document']

411       서울  고승범 금융위원장이 5일 정부서울청사에서 열린 이임식에서 이임사를 하고 있다.
829     서울  고승범 금융위원장이 5일 정부서울청사에서 열린 이임식에서 직원들로부터 박수를...
1235         서울  고승범 금융위원장이 5일 정부서울청사에서 열린 이임식에서 인사하고 있다.
1419                                 다음은 6일 장마감 후 주요 뉴스다.
3020                                 다음은 5일 장마감 후 주요 뉴스다.
3225      서울  고승범 금융위원장이 5일 정부서울청사에서 열린 이임식에서 이임사를 하고 있다.
3706      서울  고승범 금융위원장이 5일 정부서울청사에서 열린 이임식에서 이임사를 하고 있다.
3856     서울  고승범 금융위원장이 5일 정부서울청사에서 열린 이임식에서 국기에 경례하고 있다.
5991                             다음은 1일 장 마감 후 주요 종목 뉴스다.
6374          서울  고승범 금융위원장이 5일 정부서울청사에서 열린 이임식에 입장하고 있다.
6610                             다음은 1일 장 마감 후 주요 종목 뉴스다.
Name: document, dtype: object

In [ ]:
tmp.document[-3:]

7142    425
7358    641
7636    919
Name: document, dtype: int64

In [ ]:
df_val.loc[tmp.document[-3:],'document']

425     다음은 4일 장마감 후 주요 뉴스다.
641     다음은 7월4일자 신문 주요 뉴스다.
919     다음은 4일 장마감 후 주요 뉴스다.
Name: document, dtype: object

In [ ]:
# NaN 값 제거
df_cleaned = copy.deepcopy(df_loaded.drop(remove_idx_list))
df_cleaned.reset_index(drop=True,inplace=True)
df_cleaned

0       하나은행 외환시장 우리나라 94억달러 줄었다 2008년 글로벌 금융위기 한국은행 따...
1       무역적자 마이크론 반도체 쏟아져 하나은행 딜링룸 코스피 22.02 포인트 0.94%...
2       김근수 비트코인 서초구 전광판 나타나고 2022.07 com 받습니다 뉴시스 소중한...
3       에어쇼 항공기 난이도 8800 블랙이글스 항공기 에어쇼 블랙이글스 black eag...
4                                            화물차 줄여주기 보조금
                              ...                        
8137    2022 전시회 고양시 킨텍스 전시장 이정식 고용노동부 받으며 2022.07 scc...
8138    소비자 imf 외환위기 24년 치솟은 나타난 통계청 소비자 따르면 지난달 소비자 6...
8139    영등포구 여의도 3300 1000 콘셉트 포레스트 사이사이 초대형 다양한 캐릭터 했...
8140    양주시 폐기물 종량제 한다고 밝혔다 양주시 양주시 재판매 양주시 폐기물 폐기물 폐기...
8141    24일 31일 양일간 2022 스포츠 영남권 fifa 온라인 하스스톤 레전드 20일...
Name: clean_doc, Length: 8142, dtype: object

In [ ]:
tmp = copy.deepcopy(news_df.loc[remove_idx_list,['document','data type']])
tmp

,document,data type
411,411,train
829,829,train
1235,1235,train
1419,1419,train
3020,3020,train
3225,3225,train
3706,3706,train
3856,3856,train
5991,5991,train
6374,6374,train


In [ ]:
tmp = copy.deepcopy(news_df.loc[remove_idx_list,['document','data type']])
r1_list = tmp.iloc[np.where(tmp.loc[:'data type']=='train')[0],0].values.tolist()
r2_list = tmp.iloc[np.where(tmp.loc[:'data type']=='val')[0],0].values.tolist()

r1_list, r2_list

([411, 829, 1235, 1419, 3020, 3225, 3706, 3856, 5991, 6374, 6610],
 [425, 641, 919])

In [ ]:
df_train.drop(r1_list,inplace=True)
df_val.drop(r2_list,inplace=True)
df_train.shape[0] + df_val.shape[0]

8142

### LSA 시작

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [ ]:
df_cleaned

0       하나은행 외환시장 우리나라 94억달러 줄었다 2008년 글로벌 금융위기 한국은행 따...
1       무역적자 마이크론 반도체 쏟아져 하나은행 딜링룸 코스피 22.02 포인트 0.94%...
2       김근수 비트코인 서초구 전광판 나타나고 2022.07 com 받습니다 뉴시스 소중한...
3       에어쇼 항공기 난이도 8800 블랙이글스 항공기 에어쇼 블랙이글스 black eag...
4                                            화물차 줄여주기 보조금
                              ...                        
8137    2022 전시회 고양시 킨텍스 전시장 이정식 고용노동부 받으며 2022.07 scc...
8138    소비자 imf 외환위기 24년 치솟은 나타난 통계청 소비자 따르면 지난달 소비자 6...
8139    영등포구 여의도 3300 1000 콘셉트 포레스트 사이사이 초대형 다양한 캐릭터 했...
8140    양주시 폐기물 종량제 한다고 밝혔다 양주시 양주시 재판매 양주시 폐기물 폐기물 폐기...
8141    24일 31일 양일간 2022 스포츠 영남권 fifa 온라인 하스스톤 레전드 20일...
Name: clean_doc, Length: 8142, dtype: object

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 행렬 만들기
vectorizer = TfidfVectorizer(max_features=2000)  # 100개의 단어 축
X = vectorizer.fit_transform(df_cleaned)

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :', X.shape)

TF-IDF 행렬의 크기 : (8142, 2000)


In [ ]:
t_l = []
for i in range(X.shape[0]):
    if abs(X[0]).sum() == 0:
        t_l.append(i)
t_l

[]

In [ ]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

# LSA

svd_model = TruncatedSVD(n_components=10, n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

10

In [ ]:
# 문서별 토픽 분포 계산
document_topic_distributions = svd_model.transform(X)

# # 소프트맥스 함수로 정규화
# def softmax(x):
#     e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
#     return e_x / e_x.sum(axis=1, keepdims=True)

# normalized_topic_distributions = softmax(document_topic_distributions)

# 결과를 데이터프레임으로 변환
df_topics = pd.DataFrame(document_topic_distributions, columns=[f"Topic {i+1}" for i in range(10)])
df_topics.head()

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10
0,0.039990,0.101220,0.112901,0.031146,-0.047891,-0.018430,-0.028294,-0.005840,0.002452,-0.040023
1,0.135962,0.198793,0.415147,0.012801,0.042149,0.012582,-0.054892,-0.000189,-0.028134,-0.031301
2,0.725838,-0.158547,-0.041416,-0.048954,0.018857,-0.012273,-0.030503,0.005212,-0.023631,0.030791
3,0.011398,0.062760,-0.025136,-0.006605,0.008960,0.004324,-0.006508,-0.008780,0.007524,-0.004278
4,0.001330,0.008044,-0.001965,0.000570,-0.005342,0.000846,-0.003058,-0.001939,-0.002317,-0.000561


In [ ]:
# 각 문서의 토픽 분포를 하나의 컬럼으로 합치기
df_topics_combined = pd.DataFrame({
    'Topic Distribution': df_topics.values.tolist()
}, index=[f"문서 {i+1}" for i in range(len(df_topics))])

# 결과 출력
print("문서별 토픽 분포:")
print(df_topics_combined.head())

문서별 토픽 분포:
                                     Topic Distribution
문서 1  [0.03999049613183444, 0.10121976885793593, 0.1...
문서 2  [0.13596160887858635, 0.19879344594972626, 0.4...
문서 3  [0.7258381407725131, -0.15854700011110898, -0....
문서 4  [0.011397986753134401, 0.0627596939075256, -0....
문서 5  [0.001330402816273424, 0.008044336040554647, -...


In [ ]:
import copy
topic_distributions = copy.deepcopy(df_topics_combined['Topic Distribution'])
topic_distributions[1]

[0.13596160887858635,
 0.19879344594972626,
 0.415146595067185,
 0.012800889979590018,
 0.04214880029657142,
 0.012581898147570367,
 -0.05489209426331858,
 -0.00018860603874413704,
 -0.028134238457388293,
 -0.0313013767753479]

In [ ]:
topic_distributions = topic_distributions.values.tolist()

In [ ]:
train_df = pd.DataFrame({'document':df_train['document'], 'topic_distribution':topic_distributions[:len(df_train)],	'press_encoded':df_train['press_encoded']})
val_df = pd.DataFrame({'document':df_val['document'], 'topic_distribution':topic_distributions[len(df_train):],	'press_encoded':df_val['press_encoded']})

In [ ]:
train_df.head()

,document,topic_distribution,press_encoded
0,서울 5일 서울 중구 하나은행 위변조대응센터에서 직원이 달러를 정리하고 있다. 최...,"[0.03999049613183444, 0.10121976885793593, 0.1...",0
1,기사내용 요약 무역적자·경기침체 영향 마이크론發 반도체 투심 악화…삼전 매물 쏟아져...,"[0.13596160887858635, 0.19879344594972626, 0.4...",1
2,서울 김근수기자 비트코인이 경기 침체와 고금리에 대한 우려로 가상화폐 가격이 등락을...,"[0.7258381407725131, -0.15854700011110898, -0....",1
3,국제 에어쇼 참가 T 50B 항공기 9대 원주→영국 운송 완료 분해·운송·재조립 최...,"[0.011397986753134401, 0.0627596939075256, -0....",0
4,서울 고유가 상황에서 화물차·버스·택시업계의 유류비 부담을 줄여주기 위해 도입된 ...,"[0.001330402816273424, 0.008044336040554647, -...",2


In [ ]:
df_test.head()

,document,press_encoded
0,서울 롯데면세점은 지난달 내국인 매출이 지난해 같은 기간보다 약 260% 증가하는...,2
1,부산 에너지·원자재 가격 상승으로 상반기 1 6월 무역수지가 역대 최대 규모인 1...,1
2,호텔 카지노 리테일 부문 전년 대비 2배 성장 성수기 국제관광 재개로 3분기 괄목할...,0
3,1분기 KB주택구입잠재력지수 전년비 반토막 역대최저 매매가격지수 상승폭보다 주담대 ...,0
4,SSG닷컴이 서울 강남권역으로 본사 이전을 완료했다. SSG닷컴은 자회사인 W컨셉과...,4


In [ ]:
df_test['new'] = df_test['document'].apply(lambda x: svd_model.transform(vectorizer.transform([x])))
df_test['new'] = df_test['new'].apply(lambda x: x.flatten().tolist())
df_test.head()

,document,press_encoded,new
0,서울 롯데면세점은 지난달 내국인 매출이 지난해 같은 기간보다 약 260% 증가하는...,2,"[0.020986284680920158, 0.11287305674132352, -0..."
1,부산 에너지·원자재 가격 상승으로 상반기 1 6월 무역수지가 역대 최대 규모인 1...,1,"[0.5872751861895099, -0.03714790207815595, -0...."
2,호텔 카지노 리테일 부문 전년 대비 2배 성장 성수기 국제관광 재개로 3분기 괄목할...,0,"[0.02551132391794048, 0.12387364293834274, -0...."
3,1분기 KB주택구입잠재력지수 전년비 반토막 역대최저 매매가격지수 상승폭보다 주담대 ...,0,"[0.06272091926931322, 0.17120494266390254, 0.0..."
4,SSG닷컴이 서울 강남권역으로 본사 이전을 완료했다. SSG닷컴은 자회사인 W컨셉과...,4,"[0.03521384551531898, 0.1330543659032572, -0.0..."


In [ ]:
test_topic_distributions = df_test['new'].values.tolist()
len(test_topic_distributions)

1440

In [ ]:
test_df = pd.DataFrame({'document':df_test['document'], 'topic_distribution':test_topic_distributions,	'press_encoded':df_test['press_encoded']})

In [ ]:
train_df['topic_distribution'] = train_df['topic_distribution'].apply(lambda x: ','.join(map(str, x)))
val_df['topic_distribution'] = val_df['topic_distribution'].apply(lambda x: ','.join(map(str, x)))
test_df['topic_distribution'] = test_df['topic_distribution'].apply(lambda x: ','.join(map(str, x)))

In [ ]:
train_df

,document,topic_distribution,press_encoded
0,서울 5일 서울 중구 하나은행 위변조대응센터에서 직원이 달러를 정리하고 있다. 최...,"0.03999049613183444,0.10121976885793593,0.1129...",0
1,기사내용 요약 무역적자·경기침체 영향 마이크론發 반도체 투심 악화…삼전 매물 쏟아져...,"0.13596160887858635,0.19879344594972626,0.4151...",1
2,서울 김근수기자 비트코인이 경기 침체와 고금리에 대한 우려로 가상화폐 가격이 등락을...,"0.7258381407725131,-0.15854700011110898,-0.041...",1
3,국제 에어쇼 참가 T 50B 항공기 9대 원주→영국 운송 완료 분해·운송·재조립 최...,"0.011397986753134401,0.0627596939075256,-0.025...",0
4,서울 고유가 상황에서 화물차·버스·택시업계의 유류비 부담을 줄여주기 위해 도입된 ...,"0.001330402816273424,0.008044336040554647,-0.0...",2
...,...,...,...
6712,기사내용 요약 코로나 기간 마이너스 대출 수혜…ECB 이사회 소집해 논의 서울 유...,"0.020812238392610597,0.08620827633165346,0.027...",1
6713,평양 노동신문 북한 노동당 기관지 노동신문은 5일 연포온실농장 건설 현장에서 자기 ...,"0.0015584786296659785,0.006493917762268321,-0....",0
6714,서울 갤러리아백화점이 명품관에서 국내산 햇사과 썸머킹 을 올해 처음 선보인다고 3일...,"0.012504569531410586,0.07035492505748972,-0.03...",2
6715,기사내용 요약 어촌어항공단·자이언트펭tv 협업… 안전문화 확산 서울 펭수·가수 KC...,"0.012657428396420116,0.0682025163310191,-0.022...",1


In [ ]:
val_df

,document,topic_distribution,press_encoded
0,이동석 대표이사 미래 생존ㆍ고용안정 방안 찾자 현대차 노사 교섭 대표들 자료사진 울...,"0.05551473735540774,0.22053631913456231,-0.021...",2
1,기사내용 요약 블랙이글스 T 50B 항공기 9대 국제 에어쇼 참가 특수화물 장비 투...,"0.06321233830529661,0.04037834696830419,-0.027...",1
2,서울 이종호 과학기술정보통신부 장관이 5일 오후 경기도 성남시 수정구 메타버스허브에...,"0.0155349857480547,0.06698821933025828,-0.0469...",2
3,호반그룹 창립 33주년 기념식 호반그룹 제공 서울 호반그룹은 창립 33주년을 맞아...,"0.028420703894053923,0.12582526803742858,-0.04...",2
4,서울 실거주 목적의 전입신고를 했다면 재개발 사업이 진행 중인 지역이라는 이유로 ...,"0.6215638647156858,-0.11374949255915118,-0.071...",1
...,...,...,...
1434,고양 2022 국제안전보건전시회가 열린 4일 오전 경기 고양시 킨텍스 전시장에서 ...,"0.6585724542168686,-0.13542366486231736,-0.088...",1
1435,서울 국내 소비자물가 상승률이 IMF 외환위기 이후 약 24년 만에 6%대로 치솟...,"0.07917332059000258,0.15812215650485734,0.0581...",0
1436,서울 3일 서울 영등포구 여의도 더현대서울에서 열린 월리 행복 걷기 챌린지 행사에...,"0.022282577815047874,0.07211147832612158,-0.03...",0
1437,양주 경기 양주시는 전국 최초로 섬유류 폐기물 전용 종량제봉투 를 제작해 보급한다...,"0.009121744426894597,0.032155538408480205,-0.0...",2


In [ ]:
test_df

,document,topic_distribution,press_encoded
0,서울 롯데면세점은 지난달 내국인 매출이 지난해 같은 기간보다 약 260% 증가하는...,"0.020986284680920158,0.11287305674132352,-0.01...",2
1,부산 에너지·원자재 가격 상승으로 상반기 1 6월 무역수지가 역대 최대 규모인 1...,"0.5872751861895099,-0.03714790207815595,-0.041...",1
2,호텔 카지노 리테일 부문 전년 대비 2배 성장 성수기 국제관광 재개로 3분기 괄목할...,"0.02551132391794048,0.12387364293834274,-0.001...",0
3,1분기 KB주택구입잠재력지수 전년비 반토막 역대최저 매매가격지수 상승폭보다 주담대 ...,"0.06272091926931322,0.17120494266390254,0.0661...",0
4,SSG닷컴이 서울 강남권역으로 본사 이전을 완료했다. SSG닷컴은 자회사인 W컨셉과...,"0.03521384551531898,0.1330543659032572,-0.0612...",4
...,...,...,...
1435,서울 아이트로닉스. 사진 DB . photo .com 서울 4일 중소기업계 소식....,"0.10531111100003887,0.2820025215653396,-0.1209...",1
1436,서울 올 하반기 서울과 인접한 이른바 서울 옆세권 지역에서 아파트 단지들이 줄줄이...,"0.03859601024801733,0.15036835329820497,0.0108...",1
1437,취임사 하는 김기웅 제47대 서천군수 서천군 제공. 재판매 및 DB 금지 서천 김...,"0.013489350445206594,0.05554132786705248,-0.01...",2
1438,서울 조재호 농촌진흥청장이 4일 전남 무안 국립식량과학원 바이오에너지작물연구소를 방...,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0",2


In [ ]:
train_df.to_csv(path+'LSA_train_df.csv', index = False)
val_df.to_csv(path+'LSA_val_df.csv', index = False)
test_df.to_csv(path+'LSA_test_df.csv', index = False)